In [3]:
from collections import defaultdict
import math

def sent_processing(lines):

    if isinstance(lines, list):
        lines = [line.strip().split(" ") for line in lines]

        corpus = []
        for line in lines:
            sent = []
            for word in line:
                word = tuple(word.rsplit("/", 1))
                sent.append(word)
            corpus.append(sent)

        return corpus

    elif isinstance(lines, str):
        line = []
        for word in lines.strip().split(" "):
            word = tuple(word.rsplit("/", 1))
            line.append(word)
        return line

    else:
        print("wrong type of input sentence")
        exit(1)

    
with open("corpus.txt", "r", encoding='utf-8') as f:
    lines = f.readlines()

corpus = sent_processing(lines)

In [4]:
def train(corpus):

    def bigram_count(sent):
        poslist = [pos for _, pos in sent] # [NN, VBD, DT, NN]
        return [(pos0, pos1) for pos0, pos1 in zip(poslist, poslist[1:])]

    pos2words = defaultdict(lambda: defaultdict(int)) # number of (word, tag)
    trans = defaultdict(int) # bigram count --> (tag-1, tag)
    bos = defaultdict(int) # count for bos bigram --> number of (BOS, tag)

    # sent format: [(word, tag), (word, tag), ....)]
    for sent in corpus: # counting
        for word, pos in sent:
            pos2words[pos][word] +=1

        for bigram in bigram_count(sent):
            trans[bigram] +=1

        bos[sent[0][1]] +=1 # number of (BOS, tag) bigram
        trans[(sent[-1][1], 'EOS')] +=1 # number of (tag, EOS) bigram

    # obervation p(x|y) - emission prob.
    base = {pos:sum(words.values()) for pos, words in pos2words.items()} # P(y) for every y (count for each tag)
    
    # case 1
    # pos2words_prob = {pos:{word:math.log(count/base[pos]) for word, count in words.items()}
    #               for pos, words in pos2words.items()} # log(p(x, y)/p(y)) for every (x, y)
    
    # case 2
    # pos2words_prob = defaultdict(lambda: defaultdict(int))
    # for pos, words in pos2words.items():
    #     for word, count in words.items():
    #         pos2words_prob[pos][word] = math.log(count/base[pos])
    
    pos2words_ = defaultdict(lambda: defaultdict(int))
    for pos, words in pos2words.items():
        for word, count in words.items():
            pos2words_[pos][word] = math.log(count/base[pos])

    # p(y_k|p_(k-1)) - transition prob.
    base = defaultdict(int)
    for (pos0, pos1), count in trans.items():
        base[pos0] += count # p(y_(k-1))

    trans_ = {pos: math.log(count/base[pos[0]]) for pos, count in trans.items()} # p(y_k, y_(k-1))/p(y_(k-1))

   # BOS
    base = sum(bos.values()) # p(BOS)
    bos_ = {pos: math.log(count/base) for pos, count in bos.items()} # P(tag|BOS) 

    return pos2words_, trans_, bos_

In [5]:
pos2words, trans, bos = train(corpus)

print('명사 라면의 확률:', pos2words['CMC']['라면']) # 명사 '라면'의 확률 (신라면, 진라면 등.)
print('연결어미 라면의 확률:', pos2words['fmoc']['라면']) # 연결어미 '라면'의 확률 (~ 이라면)

명사 라면의 확률: -9.427948631791715
연결어미 라면의 확률: -5.6937321388027


In [18]:
class HMM_tagger(object):
    def __init__(self, pos2words, trans, bos):
        self.pos2words = pos2words
        self.trans = trans
        self.bos = bos
        self.unk = -15
        self.eos ='EOS'

    def sent_log_prob(self, sent):
        # emission prob.
        log_prob = sum(
            (self.pos2words.get(t, {}).get(w, self.unk))
            for w, t in sent
        ) # get emission prob. for each (w, t), otherwise unk value

        # bos
        log_prob += bos.get(sent[0][1], self.unk) # get BOS prob for the first (w, t)

        # transition prob.
        bigrams = [(t0, t1) for (_, t0), (_, t1) in zip(sent, sent[1:])] # every bigram in sentence
        log_prob+= sum(
            (self.trans.get(bigram, self.unk)
             for bigram in bigrams)
        )

        # eos
        log_prob += self.trans.get(
            (sent[-1][1], self.eos), self.unk)

        # length norm.
        log_prob /= len(sent)

        return log_prob


In [19]:
tagger = HMM_tagger(pos2words, trans, bos)
test_sent1= "감기/CMC 는/fjb 줄이/YBD 다/fmof ./g"
test_sent2= "감기/fmotg 는/fjb 줄/CMC 이다/fjj ./g"
print("%s: %f" % (test_sent1, tagger.sent_log_prob(sent_processing(test_sent1))))
print("%s: %f" % (test_sent2, tagger.sent_log_prob(sent_processing(test_sent2))))

감기/CMC 는/fjb 줄이/YBD 다/fmof ./g: -5.489636
감기/fmotg 는/fjb 줄/CMC 이다/fjj ./g: -14.037157
